<a href="https://colab.research.google.com/github/katL7/GTSRB-APS360/blob/main/APS360_Traffic_Sign_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Download triffic sign data
import torch
from torchvision import datasets, transforms
from google.colab import drive
import os

In [ ]:
gtsrb_data = datasets.GTSRB('data', download=True)
gtsrb_rotated = datasets.GTSRB('data', download=True, 
                           transform=transforms.RandomRotation(25))

In [ ]:
augmented_datasets = []

my_transform = transforms.Compose([
    transforms.RandomRotation(25),
    transforms.ToTensor(),
])

for _ in range(2):
    gtsrb_new = datasets.GTSRB('data', download=True, transform=my_transform)
    augmented_datasets.append(gtsrb_new)

concat = torch.utils.data.ConcatDataset(augmented_datasets)

In [ ]:
# mount our Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
SyRa_folder = "drive/MyDrive/APS360/Project/100000_nets_ema.ckpt"

SyRa = torch.load(SyRa_folder)

In [ ]:
os.chdir('drive/MyDrive/APS360/Project/SyRa-Synthesized_Rain_dataset-main')

In [ ]:
%%shell
python main.py --img_size 256 --mode syn --checkpoint_dir expr/checkpoint/SyRa --out_dir expr/result --data folder_of_your_data --resume_iter 100000

Traceback (most recent call last):
  File "main.py", line 5, in <module>
    from munch import Munch
ModuleNotFoundError: No module named 'munch'


CalledProcessError: ignored

In [1]:
# Test edit